In [1]:
%load_ext autoreload

# Automatically reload all modules (except those excluded by %aimport)
%autoreload 2

In [2]:
import pandas as pd
from pydantic import BaseModel, model_validator, field_validator, Field, ValidationInfo, Extra
from typing import List, Dict, Union, Any, Optional, Literal
import instructor
from openai import OpenAI
import os
import json
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.documents.base import Document
from langchain_openai import OpenAIEmbeddings

from wikidata_search import WikidataSearch, get_all_properties_with_labels
from data_classes import KnowledgeGraph, ValidatedProperty
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain.tools import DuckDuckGoSearchRun
import sys
sys.path.insert(0, '../../knowledge_graph_validator')
import utils
import validators

In [12]:
DATASET_NAME = 'FB13'

save_path = f'../../data/web_context_{DATASET_NAME}_evaluation_results.jsonl'

In [13]:
client = instructor.patch(OpenAI(api_key=os.environ['OPENAI_API_KEY']))
MODEL = "gpt-3.5-turbo-0125"

# 🛜 Web Search 

In [14]:
# https://github.com/deedy5/duckduckgo_search?tab=readme-ov-file#TOP
from duckduckgo_search import DDGS

results = DDGS().text("Umberto I King of Italy cause of death?", max_results=3)
for r in results: print(r)

{'title': 'Umberto I of Italy - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Umberto_I_of_Italy', 'body': "Umberto I (Italian: Umberto Rainerio Carlo Emanuele Giovanni Maria Ferdinando Eugenio di Savoia; 14 March 1844 - 29 July 1900) was King of Italy from 9 January 1878 until his assassination in 1900. His reign saw Italy's expansion into the Horn of Africa, as well as the creation of the Triple Alliance among Italy, Germany and Austria-Hungary.. The son of Victor Emmanuel II and Adelaide of ..."}
{'title': 'Italian American assassinates Italian king | July 29, 1900 | HISTORY', 'href': 'https://www.history.com/this-day-in-history/italian-american-assassinates-italian-king', 'body': 'In Monza, Italy, King Umberto I is shot to death by Gaetano Bresci, an Italian-born anarchist who resided in America before returning to his homeland to murder the king. Crowned in 1878, King ...'}
{'title': 'This Is The Assassination Of King Umberto Of Italy Explained', 'href': 'https://www.grunge.c

# 🧠 Load data

In [15]:
'''datasets: FB13, WN11, WN18RR, YAGO3-10'''
positive_triples, negative_triples = utils.read_dataset(DATASET_NAME)

print("Positive Triples:")
for triple in positive_triples[:6]:
    print(triple)

print("\nNegative Triples:")
for triple in negative_triples[:6]:
    print(triple)

print(len(positive_triples) + len(negative_triples))

Positive Triples:
{'subject': 'Umberto I Of Italy', 'relation': 'cause of death', 'object': 'Tyrannicide'}
{'subject': 'John Glenn Beall Jr', 'relation': 'nationality', 'object': 'United States'}
{'subject': 'John Atkinson Grimshaw', 'relation': 'gender', 'object': 'Male'}
{'subject': 'Hardinge Giffard 1St Earl Of Halsbury', 'relation': 'gender', 'object': 'Male'}
{'subject': 'Mike Von Erich', 'relation': 'nationality', 'object': 'United States'}
{'subject': 'Gilbert Carlton Walker', 'relation': 'nationality', 'object': 'United States'}

Negative Triples:
{'subject': 'Umberto I Of Italy', 'relation': 'cause of death', 'object': 'Cerebral Aneurysm'}
{'subject': 'John Glenn Beall Jr', 'relation': 'nationality', 'object': 'Ancient Greece'}
{'subject': 'John Atkinson Grimshaw', 'relation': 'gender', 'object': 'Female'}
{'subject': 'Hardinge Giffard 1St Earl Of Halsbury', 'relation': 'gender', 'object': 'Female'}
{'subject': 'Mike Von Erich', 'relation': 'nationality', 'object': 'Serbia'}
{

# 🪬 Define Evaluation Model

In [18]:
idx = 7
subject, relation, object = negative_triples[idx]['subject'], negative_triples[idx]['relation'], negative_triples[idx]['object']

search_query = validators.WebKGValidator.create_query(subject, relation, object)

search_tool = DDGS()
web_reference = validators.WebKGValidator.get_web_search_results(search_tool, search_query)

print(search_query)
print(web_reference)

What Phil Walden gender?
[{'title': 'Phil Walden - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Phil_Walden', 'body': "Phil Walden (January 11, 1940 - April 23, 2006) was a co-founder of the Macon, Georgia-based Capricorn Records, along with former Atlantic Records executive Frank Fenter. Biography. Walden received his undergraduate degree in economics from Macon's Mercer University (where he was a member of Phi Delta Theta and a ROTC cadet) in 1962."}, {'title': 'Southern rock pioneer Phil Walden dies. Under Capricorn Records, he ...', 'href': 'https://www.rollingstone.com/music/music-news/southern-rock-pioneer-walden-dies-87776/', 'body': 'Music impresario Phil Walden, who managed Otis Redding and helped define Southern rock through his work with the Allman Brothers, the Charlie Daniels Band, the Marshall Tucker Band and many other ...'}, {'title': 'Phil Walden, 66; Ran Capricorn Records, Home to Allman Brothers ...', 'href': 'https://www.latimes.com/archives/la-xpm-2006-apr-25

# Evaluate!

In [8]:
neg_results = []

neg_results.append(validators.WebKGValidator(**{'triples': negative_triples[:7]}))

neg_results[0].model_dump()['validated_triples']

[{'entity_label': 'umberto_i_of_italy',
  'property_name': 'cause_of_death',
  'property_value': 'cerebral_aneurysm',
  'property_is_valid': False,
  'is_valid_reason': None,
  'error_message': 'The predicted cause of death for Umberto I of Italy as cerebral aneurysm is inaccurate based on the provided context. Umberto I was actually assassinated by Gaetano Bresci, an Italian-born anarchist, in 1900. The historical and factual validity does not align with the predicted property value.',
  'sources': [[{'title': 'Umberto I of Italy - Wikipedia',
     'href': 'https://en.wikipedia.org/wiki/Umberto_I_of_Italy',
     'body': "Umberto I (Italian: Umberto Rainerio Carlo Emanuele Giovanni Maria Ferdinando Eugenio di Savoia; 14 March 1844 - 29 July 1900) was King of Italy from 9 January 1878 until his assassination in 1900. His reign saw Italy's expansion into the Horn of Africa, as well as the creation of the Triple Alliance among Italy, Germany and Austria-Hungary.. The son of Victor Emmanue

In [9]:
pos_results = []

pos_results.append(validators.WebKGValidator(**{'triples': positive_triples[:7]}))

pos_results[0].model_dump()['validated_triples']

[{'entity_label': 'umberto_i_of_italy',
  'property_name': 'cause_of_death',
  'property_value': 'tyrannicide',
  'property_is_valid': False,
  'is_valid_reason': None,
  'error_message': "The predicted property value 'tyrannicide' is not accurate based on the context provided. King Umberto I of Italy was assassinated, not killed through tyrannicide. The term 'tyrannicide' refers to the act of killing a tyrant or oppressive ruler, which does not align with the historical event of King Umberto I's assassination by an anarchist.",
  'sources': [[{'title': 'Umberto I of Italy - Wikipedia',
     'href': 'https://en.wikipedia.org/wiki/Umberto_I_of_Italy',
     'body': "Umberto I (Italian: Umberto Rainerio Carlo Emanuele Giovanni Maria Ferdinando Eugenio di Savoia; 14 March 1844 - 29 July 1900) was King of Italy from 9 January 1878 until his assassination in 1900. His reign saw Italy's expansion into the Horn of Africa, as well as the creation of the Triple Alliance among Italy, Germany and 

In [10]:
tp = 0
fp = 0
tn = 0
fn = 0 
for val in pos_results[0].model_dump()['validated_triples']:
    if val['property_is_valid']:    # property is correctly marked as valid
        tp += 1
    else:                           # property is incorrectly marked as invalid
        fn += 1
for val in neg_results[0].model_dump()['validated_triples']:
    if val['property_is_valid']:    # property is incorrectly marked as valid
        fp += 1
    else:                           # property is correctly marked as invalid
        tn += 1

metrics = utils.calc_metrics(tp, fp, tn, fn)

Precision: 1.0
Recall: 0.8571428571428571
F1 Score: 0.923076923076923
Accuracy: 0.9285714285714286
----------


In [11]:
results_json = [r.model_dump() for r in neg_results] + [r.model_dump() for r in pos_results]
utils.save_jsonl(results_json, save_path)

Saved to f'../../data/web_context_FB13_evaluation_results.jsonl
